In [1]:
import torch
import numpy as np
import pandas as pd
import matchzoo as mz
print('matchzoo version', mz.__version__)

matchzoo version 1.1.1


In [2]:
ranking_task = mz.tasks.Ranking(losses=mz.losses.RankHingeLoss())
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]
print("`ranking_task` initialized with metrics", ranking_task.metrics)

`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]


In [3]:
print('data loading ...')
train_pack_raw = mz.datasets.wiki_qa.load_data('train', task=ranking_task)
dev_pack_raw = mz.datasets.wiki_qa.load_data('dev', task=ranking_task, filtered=True)
test_pack_raw = mz.datasets.wiki_qa.load_data('test', task=ranking_task, filtered=True)
print('data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`')

data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [4]:
preprocessor = mz.preprocessors.BasicPreprocessor(
    truncated_length_left = 10,
    truncated_length_right = 100,
    filter_low_freq = 2
)

In [5]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 11001.23it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 5972.72it/s]
Building Vocabulary from a datapack.: 100%|██████████| 404432/404432 [00:00<00:00, 3237271.33it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 11244.91it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 5965.35it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 908091.90it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 10722.40it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00

In [6]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x12ed4e898>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x1307ec748>,
 'vocab_size': 16675,
 'embedding_input_dim': 16675}

In [7]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=100)
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [8]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1,
    batch_size=20,
    resample=True,
    sort=False
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed,
    batch_size=20
)

In [9]:
padding_callback = mz.models.DRMMTKS.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    stage='train',
    callback=padding_callback
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    stage='dev',
    callback=padding_callback
)

In [10]:
model = mz.models.DRMMTKS()

model.params['task'] = ranking_task
model.params['embedding'] = embedding_matrix
model.params['mask_value'] = 0
model.params['top_k'] = 10
model.params['mlp_activation_func'] = 'tanh'

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

DRMMTKS(
  (embedding): Embedding(16675, 100, padding_idx=0)
  (attention): Attention(
    (linear): Linear(in_features=100, out_features=1, bias=False)
  )
  (mlp): Sequential(
    (0): Sequential(
      (0): Linear(in_features=10, out_features=128, bias=True)
      (1): Tanh()
    )
    (1): Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): Tanh()
    )
    (2): Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): Tanh()
    )
    (3): Sequential(
      (0): Linear(in_features=128, out_features=1, bias=True)
      (1): Tanh()
    )
  )
  (out): Linear(in_features=1, out_features=1, bias=True)
)
Trainable params:  1702163


In [11]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [12]:
trainer.run()

[Iter-102 Loss-0.817]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5808 - normalized_discounted_cumulative_gain@5(0.0): 0.6451 - mean_average_precision(0.0): 0.6026



[Iter-204 Loss-0.541]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5919 - normalized_discounted_cumulative_gain@5(0.0): 0.6427 - mean_average_precision(0.0): 0.6097



[Iter-306 Loss-0.403]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6074 - normalized_discounted_cumulative_gain@5(0.0): 0.6558 - mean_average_precision(0.0): 0.6144



[Iter-408 Loss-0.300]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5891 - normalized_discounted_cumulative_gain@5(0.0): 0.6367 - mean_average_precision(0.0): 0.5983



[Iter-510 Loss-0.240]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.57 - normalized_discounted_cumulative_gain@5(0.0): 0.6332 - mean_average_precision(0.0): 0.5815



[Iter-612 Loss-0.184]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5711 - normalized_discounted_cumulative_gain@5(0.0): 0.6315 - mean_average_precision(0.0): 0.5864



[Iter-714 Loss-0.136]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5783 - normalized_discounted_cumulative_gain@5(0.0): 0.6402 - mean_average_precision(0.0): 0.5988



[Iter-816 Loss-0.109]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5713 - normalized_discounted_cumulative_gain@5(0.0): 0.6242 - mean_average_precision(0.0): 0.5819



[Iter-918 Loss-0.087]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5872 - normalized_discounted_cumulative_gain@5(0.0): 0.6456 - mean_average_precision(0.0): 0.5928



[Iter-1020 Loss-0.092]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5799 - normalized_discounted_cumulative_gain@5(0.0): 0.6441 - mean_average_precision(0.0): 0.5929

Cost time: 133.27396488189697s
